In [30]:
from lxml import etree

In [31]:
with open("badhtml.html", 'r') as f:
    root = etree.HTML(f.read())

In [32]:
def attrs(node):
    if not node.attrib:
        return ""
    else:
        return " " + " ".join('%s="%s"'%it for it in node.attrib.items())
    
def txt(node):
    t = node.text or ""
    s = t.strip()
    if t and s:
        return t
    else:
        return s

def dump(node, level, out):
    ch = node.getchildren()
    space = "  " * level
    if ch:
        out("%s<%s%s>%s"%(space, node.tag, attrs(node), txt(node).strip("\r\n")))
        for n in ch:
            dump(n, level+1, out)
        out("%s<%s/>"%(space, node.tag))
    else:
        if not node.text:
            out("%s<%s%s/>"%(space, node.tag, attrs(node)))
        else:
            out("%s<%s%s>%s<%s/>"%(space, node.tag, attrs(node), txt(node), node.tag))

In [33]:
class PrintOut():
    def __init__(self, file):
        self.file = file
        self.fp = None
    def __call__(self, text):
        print(text, file=self.fp)
    def __enter__(self):
        self.fp = open(self.file, 'w')
        self.fp.__enter__()
        return self
    def __exit__(self, *args):
        self.fp.__exit__()
        self.fp = None
        
def printout(file):
    return PrintOut(file)

with PrintOut("test.html") as pout:
    pout("\n<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Strict//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd\">")
    dump(root, 0, pout)

In [36]:
for tr in root.xpath("//tr[td]"):
    for td in tr.find("//td/"):
        print(td.text or "", end="")
    print()

Column 1



















            



















In [ ]:
import pandas as pd
tables = pd.read_html(s)

In [ ]:
df.to_excel("test.xlsx", header=False, index=False, merge_cells=True, verbose=True)

In [ ]:
df = tables[0]
df.to_excel?

In [3]:
from officelib.xllib import *
xl = Excel()

In [28]:
import time

wb = xl.ActiveWorkbook
ws = wb.Worksheets(1)
columns = ws.UsedRange.Columns

def highlight(c):
    c.Interior.Color=65535
    
def dehighlight(c):
    c.Interior.Pattern=xlc.xlNone

def rowgetter(rows):
    memo = {}
    def getrow(i):
        nonlocal memo
        ret = memo.get(i, None)
        if ret is None:
            c = rows(i)
            ret = c, c.Value2
            memo[i] = ret
        return ret
    return getrow
print("start")  
cell_range = ws.Cells.Range  
for column in ws.UsedRange.Columns:
    Rows = column.Rows
    getrow = rowgetter(Rows)

    row = 1
    rc = Rows.Count
    while row <= rc:
        fc,fv = getrow(row)
        sr = row
        c, v = fc, fv
        while v == fv:
            highlight(c)
            #time.sleep(.05)
            row += 1
            if not v or row > rc:
                break
            c, v = getrow(row)

        row -= 1

        if row != sr:
            c, _ = getrow(row)
            xl.DisplayAlerts = False
            cell_range(fc, c).Merge()
            fc.VerticalAlignment = xlc.xlTop
            xl.DisplayAlerts = True
        dehighlight(cell_range(fc, c))
        row += 1
print("done")

start
done
